In [1]:
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import display
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
import yfinance as yf
import investpy
import time
from datetime import datetime, timezone

In [4]:
%matplotlib inline
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_seq_items', 500)

# High Dividend Yield Stocks

https://finance.yahoo.com/u/yahoo-finance/watchlists/high-yield-dividend-stocks/

In [5]:

from dataclasses import dataclass

@dataclass
class DividendStock:
    symbol: str = ""
    name: str = ""
    sector: str = ""
    exchange: str = ""
    currentPrice: float = 0
    earningsDate: datetime = datetime.min
    exDividendDate: datetime = datetime.min
    dividendDate: datetime = datetime.min
    dividendHistory: pd.DataFrame = pd.DataFrame()

In [ ]:
symbols = ["PBR-A", "PBR", "SNPTY", "BNPQY", "NRDBY", "AMKBY", "ISNPY", "EC", "IEP", "NLY", "AGNC", "BAK", "ELP", "ZIM", "CCU", "SBLK", "CLM", "GOGL", "RC", "CIM", "ARI", "OXLC", "MFA", "CRF", "BRMK", "EGLE", "LPG", "IVR", "ORC", "BPT"]

In [24]:
dividend_stocks = []
for symbol in symbols:
    print(f"Checking {symbol}")
    
    stock = DividendStock()
    
    ticker = yf.Ticker(symbol)
    
    stock_info = ticker.info
    stock.symbol = symbol
    stock.name = stock_info["shortName"]
    stock.sector = stock_info["sector"]
    stock.exchange = stock_info["exchange"]
    stock.currentPrice = stock_info["currentPrice"]
   
    ticker = yf.Ticker(symbol)
    stock_stats = ticker.stats()
    calendar_events = stock_stats["calendarEvents"]
    if len(calendar_events["earnings"]["earningsDate"]) > 0:
        stock.earningsDate = datetime.fromtimestamp(calendar_events["earnings"]["earningsDate"][0])
   
    if calendar_events["exDividendDate"] is not None:
        stock.exDividendDate = datetime.fromtimestamp(calendar_events["exDividendDate"])
    
    if calendar_events["dividendDate"] is not None:
        stock.dividendDate = datetime.fromtimestamp(calendar_events["dividendDate"])
    
    try:
        stock.dividendHistory = investpy.get_stock_dividends(symbol, "united states")
    except Exception as e:
        print("Unable to get dividend history", e)
   
    dividend_stocks.append(stock)
    time.sleep(2)

Checking CEN
Unable to get dividend history ERR#0018: stock cen not found, check if it is correct.
Checking CHKR
Unable to get dividend history ERR#0018: stock chkr not found, check if it is correct.
Checking NTG
Unable to get dividend history ERR#0018: stock ntg not found, check if it is correct.
Checking FMO
Unable to get dividend history ERR#0018: stock fmo not found, check if it is correct.
Checking CLM
Unable to get dividend history ERR#0061: introduced stock has no dividend's data to display.
Checking CRF
Unable to get dividend history ERR#0061: introduced stock has no dividend's data to display.
Checking OXLC
Unable to get dividend history ERR#0018: stock oxlc not found, check if it is correct.
Checking IVR
Checking ORC
Unable to get dividend history ERR#0018: stock orc not found, check if it is correct.
Checking ECTM
Unable to get dividend history ERR#0018: stock ectm not found, check if it is correct.
Checking GER
Unable to get dividend history ERR#0018: stock ger not found, c

In [26]:
for stock in dividend_stocks:
    if stock.earningsDate.year == 2022 and stock.earningsDate.month == 10:
        print(f"** {stock.name}  -  {stock.symbol}") 
        print(f"Sector  -  {stock.sector}")
        print(f"Exchange  -  {stock.exchange}")
        print(f"Earning Date  -  {stock.earningsDate}")
        print(f"Ex Dividend Date  -  {stock.exDividendDate}")
        print(f"Dividend Date  -  {stock.dividendDate}")
        print(f"Current Price - {stock.currentPrice}")
        print(stock.dividendHistory.head(10))
        print("\n")

** Oxford Lane Capital Corp.  -  OXLC
Sector  -  Financial Services
Exchange  -  NGM
Earning Date  -  2022-10-27 08:00:00
Ex Dividend Date  -  2022-08-16 08:00:00
Dividend Date  -  2019-03-29 08:00:00
Current Price - 5.52
Empty DataFrame
Columns: []
Index: []


** Orchid Island Capital, Inc.  -  ORC
Sector  -  Real Estate
Exchange  -  NYQ
Earning Date  -  2022-10-26 08:00:00
Ex Dividend Date  -  2022-07-28 08:00:00
Dividend Date  -  2022-10-27 08:00:00
Current Price - 9.12
Empty DataFrame
Columns: []
Index: []


** Martin Midstream Partners L.P.  -  MMLP
Sector  -  Energy
Exchange  -  NMS
Earning Date  -  2022-10-18 08:00:00
Ex Dividend Date  -  2022-08-04 08:00:00
Dividend Date  -  2022-08-12 08:00:00
Current Price - 3.58
        Date  Dividend       Type Payment Date   Yield
0 2022-08-04    0.0050  quarterly   2022-08-12   0.45%
1 2022-05-05    0.0050  quarterly   2022-05-13   0.39%
2 2022-02-04    0.0050  quarterly   2022-02-14   0.64%
3 2021-11-04    0.0050  quarterly   2021-11-12 

# Analyse High Dividend Stocks

In [6]:
def from_epoch_time(value) -> datetime:
    return pd.to_datetime(value, unit="s")

In [7]:
COUNTRY = "united states"

STOCK_FILE = "data/stock.xlsx"
SHEET_NAME_STOCK = "stock"

STOCK_INFO_FILE = "data/stock_info.xlsx"
SHEET_NAME_STOCK_INFO = "stock_info"

STOCK_STATS_FILE = "data/stock_stats.xlsx"
SHEET_NAME_STOCK_STATS = "stock_stats"

STOCK_DIVIDEND_FILE = "data/stock_dividends.xlsx"
SHEET_NAME_STOCK_DIVIDEND = "stock_dividends"

In [8]:
df_stock = pd.read_excel(STOCK_FILE, sheet_name=SHEET_NAME_STOCK, engine="openpyxl")
df_stock_info = pd.read_excel(STOCK_INFO_FILE, sheet_name=SHEET_NAME_STOCK_INFO, engine="openpyxl")
df_stock_stats = pd.read_excel(STOCK_STATS_FILE, sheet_name=SHEET_NAME_STOCK_STATS, engine="openpyxl")
df_stock_dividend = pd.read_excel(STOCK_DIVIDEND_FILE, sheet_name=SHEET_NAME_STOCK_DIVIDEND, engine="openpyxl")

In [9]:
# Convert field names to lower case
df_stock = df_stock.rename(columns=str.lower)
df_stock_info = df_stock_info.rename(columns=str.lower)
df_stock_stats = df_stock_stats.rename(columns=str.lower)
df_stock_dividend = df_stock_dividend.rename(columns=str.lower)

In [10]:
df_stock_stats["lastdividenddate"] = df_stock_stats["lastdividenddate"].apply(from_epoch_time)

In [11]:
df_yield = df_stock_stats.sort_values(by=["fiveyearavgdividendyield"], ascending=False)[["symbol","name","sector","fiveyearavgdividendyield", "lastdividendvalue", "lastdividenddate", "threeyearaveragereturn", "fiveyearaveragereturn" ]]

In [16]:
df_high_yield = df_yield[df_yield["lastdividenddate"].dt.year == 2022]
df_high_yield.head(15)

,symbol,name,sector,fiveyearavgdividendyield,lastdividendvalue,lastdividenddate,threeyearaveragereturn,fiveyearaveragereturn
1439,CEN,Center Coast Brookfield MLP & E,Financial Services,24.95,0.225,2022-09-13,NaN,NaN
1530,CHKR,CHESAPEAKE GRANITE WASH TR,Energy,22.62,0.085,2022-08-18,NaN,NaN
5335,NTG,"Tortoise Midstream Energy Fund,",Financial Services,20.49,0.770,2022-08-23,NaN,NaN
2893,FMO,Fiduciary/Claymore Energy Infra,Financial Services,20.12,0.245,2022-02-14,NaN,NaN
1617,CLM,111422,Financial Services,19.93,0.181,2022-09-14,NaN,NaN
1828,CRF,33138,Financial Services,19.61,0.173,2022-09-14,NaN,NaN
5632,OXLC,Oxford Lane Capital Corp.,Financial Services,18.62,0.075,2022-09-15,NaN,NaN
4092,IVR,INVESCO MORTGAGE CAPITAL INC,Real Estate,18.20,0.900,2022-07-08,NaN,NaN
5575,ORC,"Orchid Island Capital, Inc.",Real Estate,18.03,0.160,2022-08-30,NaN,NaN
2340,ECTM,ECA Marcellus Trust I,Energy,17.98,0.176,2022-08-18,NaN,NaN


In [23]:
symbols = df_high_yield["symbol"].head(15).tolist()

# By Sector

In [46]:
df_yield_sector = df_stock_stats.sort_values(by=["sector", "fiveyearavgdividendyield"], ascending=False)[["symbol","name", "sector", "fiveyearavgdividendyield", "lastdividendvalue", "lastdividenddate" ]]
df_yield_sector = df_yield_sector[ (df_yield_sector["lastdividenddate"].dt.year == 2022) & (df_yield_sector["fiveyearavgdividendyield"].notnull())]

In [47]:
df_yield_sector.groupby(by=["sector"]).head(2)

,symbol,name,sector,fiveyearavgdividendyield,lastdividendvalue,lastdividenddate
6937,SPH,"Suburban Propane Partners, L.P.",Utilities,11.04,0.3250,2022-08-01
3290,GNE-PA,Genie Energy Ltd. Series 2012 -,Utilities,7.67,0.1594,2022-08-05
4813,MINDP,"MIND Technology, Inc. - Series",Technology,11.15,0.5630,2022-04-13
5378,NVEC,NVE Corporation,Technology,5.85,1.0000,2022-07-29
4092,IVR,INVESCO MORTGAGE CAPITAL INC,Real Estate,18.20,0.9000,2022-07-08
5575,ORC,"Orchid Island Capital, Inc.",Real Estate,18.03,0.1600,2022-08-30
2002,CWPS,CONWEST PARTNERSHIP PART UNITS,Industrials,15.42,1.5440,2022-07-14
7684,USDP,USD Partners LP,Industrials,14.97,0.1240,2022-08-02
5003,MTBCP,"CareCloud, Inc. - 11% Series A",Healthcare,10.34,0.2290,2022-08-30
2020,CYCCP,"Cyclacel Pharmaceuticals, Inc.",Healthcare,9.30,0.1500,2022-07-14


Stocks to track
- PBR-A
- PBR
- GOGL
- ORC - monthly
- OXLC - monthly